In [1]:
from pprint import pprint

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

%matplotlib inline
import matplotlib.pyplot as plt

from prepare import basic_clean, lemmatize

import unicodedata
import re
import json
from time import strftime

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from env import get_db_url

from acquire import new_spam_data

In [5]:
df = new_spam_data()
df

,id,label,text
0,0,ham,"Go until jurong point, crazy.. Available only ..."
1,1,ham,Ok lar... Joking wif u oni...
2,2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,3,ham,U dun say so early hor... U c already then say...
4,4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...,...
5567,5567,spam,This is the 2nd time we have tried 2 contact u...
5568,5568,ham,Will Ì_ b going to esplanade fr home?
5569,5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,5570,ham,The guy did some bitching but I acted like i'd...


In [2]:
ADDITIONAL_STOPWORDS = ['r', 'u', '2', '4', 'ltgt']

In [3]:
def clean(text):
    '''
    A simple function to cleanup text data.
    
    Args:
        text (str): The text to be cleaned.
        
    Returns:
        list: A list of lemmatized words after cleaning.
    '''
    
    # basic_clean() function from last lesson:
    # Normalize text by removing diacritics, encoding to ASCII, decoding to UTF-8, and converting to lowercase
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    
    # Remove punctuation, split text into words
    words = re.sub(r'[^\w\s]', '', text).split()
    
    
    # lemmatize() function from last lesson:
    # Initialize WordNet lemmatizer
    wnl = nltk.stem.WordNetLemmatizer()
    
    # Combine standard English stopwords with additional stopwords
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    
    # Lemmatize words and remove stopwords
    cleaned_words = [wnl.lemmatize(word) for word in words if word not in stopwords]
    
    return cleaned_words

In [6]:


df['clean_text'] = df.text.apply(clean).apply(' '.join)



In [7]:
X = df.clean_text
y = df.label
X_train, X_test, y_train, y_test = \
train_test_split(X, y, 
                 test_size=0.2, 
                 random_state=1349)

In [11]:
cv = CountVectorizer()
X_bow = cv.fit_transform(X_train)
tree = RandomForestClassifier(max_depth=3)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train)


0.8714381871213821

In [12]:
X_test_bow = cv.transform(X_test)
tree.score(X_test_bow, y_test)


0.8439461883408071

In [13]:


y_train.value_counts(normalize=True)



ham     0.871438
spam    0.128562
Name: label, dtype: float64

In [14]:
pd.Series(
    dict(
    zip(cv.get_feature_names_out(), 
    tree.feature_importances_))).sort_values(ascending=False).head()


mobile     0.035553
claim      0.022353
txt        0.021953
a5000      0.021171
service    0.020584
dtype: float64